# Altair plots for the front-end

In [1]:
import altair as alt
from pathlib import Path
import pandas as pd
import numpy as np
import requests

In [2]:
HOME = Path.cwd().parent
DATA = HOME / "data" / "raw"

## London Chart

In [3]:
filename = "phe_cases_london_england.csv"
df = pd.read_csv(DATA / filename)

In [4]:
df_london = df.loc[df["area_name"] == "London"]
df_london["date"] = pd.to_datetime(df_london["date"])

/home/alangel/anaconda3/envs/nasa/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
brush = alt.selection(type="interval", encodings=["x"])

highlight = alt.selection(type="single", on="mouseover", fields=["date"], nearest=True)

base = (
    alt.Chart(df_london)
    .mark_bar(color="#1e88e5")
    .encode(alt.X("date:T"), alt.Y("new_cases:Q"))
    .properties(width=800, height=250)
)

upper = (
    base.encode(
        alt.X(
            "date:T",
            scale=alt.Scale(domain=brush),
            axis=alt.Axis(labels=False, title=" "),
        ),
        alt.Y("new_cases:Q", axis=alt.Axis(title="Daily number of cases")),
        color=alt.condition(~highlight, alt.value("#1e88e5"), alt.value("#b1010f")),
        size=alt.condition(~highlight, alt.value(3), alt.value(6)),
        tooltip=[
            alt.Tooltip("area_name:N", title="Area Name"),
            alt.Tooltip("date:T", title="Date"),
            alt.Tooltip("new_cases:Q", title="Daily new cases"),
            alt.Tooltip("total_cases:Q", title="Total cases"),
        ],
    )
    .properties(title="COVID-19 impact in London Area")
    .add_selection(highlight)
)

lower = (
    base.encode(
        alt.X("date:T", axis=alt.Axis(title=" ")),
        alt.Y("new_cases:Q", axis=alt.Axis(title=" ")),
        color=alt.condition(brush, alt.value("#1e88e5"), alt.value("lightgray")),
    )
    .properties(height=60)
    .add_selection(brush)
)

upper & lower

alt.VConcatChart(...)

## NYC Chart

In [6]:
filename = "COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv"
df_ny = pd.read_csv(DATA / filename)

In [7]:
df_ny["DATE_OF_INTEREST"] = pd.to_datetime(df_ny["DATE_OF_INTEREST"])
df_ny["Total_count"] = np.cumsum(df_ny["CASE_COUNT"])
df_ny["area_name"] = "New York City"

In [8]:
brush = alt.selection(type="interval", encodings=["x"])

highlight = alt.selection(
    type="single", on="mouseover", fields=["DATE_OF_INTEREST"], nearest=True
)

base = (
    alt.Chart(df_ny)
    .mark_bar(color="#1e88e5")
    .encode(alt.X("DATE_OF_INTEREST:T"), alt.Y("CASE_COUNT:Q"))
    .properties(width=800, height=250)
)

upper = (
    base.encode(
        alt.X(
            "DATE_OF_INTEREST:T",
            scale=alt.Scale(domain=brush),
            axis=alt.Axis(labels=False, title=" "),
        ),
        alt.Y("CASE_COUNT:Q", axis=alt.Axis(title="Daily number of cases")),
        color=alt.condition(~highlight, alt.value("#1e88e5"), alt.value("#b1010f")),
        size=alt.condition(~highlight, alt.value(3), alt.value(6)),
        tooltip=[
            alt.Tooltip("area_name:N", title="Area Name"),
            alt.Tooltip("DATE_OF_INTEREST:T", title="Date"),
            alt.Tooltip("CASE_COUNT:Q", title="Daily new cases"),
            alt.Tooltip("Total_count:Q", title="Total cases"),
        ],
    )
    .properties(title="COVID-19 impact in New York City Area")
    .add_selection(highlight)
)

lower = (
    base.encode(
        alt.X("DATE_OF_INTEREST:T", axis=alt.Axis(title=" ")),
        alt.Y("CASE_COUNT:Q", axis=alt.Axis(title=" ")),
        color=alt.condition(brush, alt.value("#1e88e5"), alt.value("lightgray")),
    )
    .properties(height=60)
    .add_selection(brush)
)

upper & lower

alt.VConcatChart(...)

## Hubei plot

In [9]:
r_china = requests.get("https://api.covid19api.com/dayone/country/china")

In [10]:
df_wuhan = pd.DataFrame(r_china.json())
df_wuhan = df_wuhan.loc[df_wuhan["Province"] == "Hubei"]
df_wuhan["daily_Confirmed"] = df_wuhan["Confirmed"].diff()
df_wuhan = df_wuhan.fillna(0)
df_wuhan["Date"] = pd.to_datetime(df_wuhan["Date"])
df_wuhan["Date"] = df_wuhan["Date"].dt.tz_convert(None)

In [11]:
brush = alt.selection(type="interval", encodings=["x"])

highlight = alt.selection(type="single", on="mouseover", fields=["Date"], nearest=True)

base = (
    alt.Chart(df_wuhan)
    .mark_bar(color="#1e88e5")
    .encode(alt.X("Date:T"), alt.Y("daily_Confirmed:Q"))
    .properties(width=800, height=250)
)

upper = (
    base.encode(
        alt.X(
            "Date:T",
            scale=alt.Scale(domain=brush),
            axis=alt.Axis(labels=False, title=" "),
        ),
        alt.Y("daily_Confirmed:Q", axis=alt.Axis(title="Daily number of cases")),
        color=alt.condition(~highlight, alt.value("#1e88e5"), alt.value("#b1010f")),
        size=alt.condition(~highlight, alt.value(3), alt.value(6)),
        tooltip=[
            alt.Tooltip("area_name:N", title="Area Name"),
            alt.Tooltip("Date:T", title="Date"),
            alt.Tooltip("daily_Confirmed:Q", title="Daily new cases"),
            alt.Tooltip("Confirmed:Q", title="Total cases"),
        ],
    )
    .properties(title="COVID-19 impact in Hubei Area")
    .add_selection(highlight)
)

lower = (
    base.encode(
        alt.X("Date:T", axis=alt.Axis(title=" ")),
        alt.Y("daily_Confirmed:Q", axis=alt.Axis(title=" ")),
        color=alt.condition(brush, alt.value("#1e88e5"), alt.value("lightgray")),
    )
    .properties(height=60)
    .add_selection(brush)
)

upper & lower

alt.VConcatChart(...)